In [8]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 25 10:40:42 2020

@author: mwall
"""
import os
import numpy as np
import pandas as pd
os.chdir('E:\ISB work stuff\Post-Doc_BaligaLab_GBMSYGNALProject')
import miner as miner

# Path to the miner directory
input_path = os.getcwd()

#expressionFile1 = os.path.join(os.path.expanduser('~'),'mnt/omics/SYGNAL/MINER/data/expression/IA12Zscore.csv')
#expressionData, conversionTable = miner.preprocess(expressionFile1)
expressionData = pd.read_csv("Rembrandt_MinerInputFiltered.csv", sep=",", index_col=0, header = 0)

# Load resulonModules
regulonModules = miner.read_json(os.path.join(input_path,"GBM\\results\\MINER\\MINER_MicroLowessRNATMM.08.24.2020\\regulons.json"))
regulonDfMicroGbmLatest = pd.read_csv("regulonDfGbmMicroRNASigCoxAndStatSig.csv", header = 0)
regulonDfMicroGbmLatest = list(regulonDfMicroGbmLatest['Regulon_ID'].drop_duplicates())
regulonDfMicroGbmLatest = [str(i) for i in regulonDfMicroGbmLatest]
regulonModules = dict((k, regulonModules[k]) for k in regulonDfMicroGbmLatest if k in regulonModules)

# Load transcriptional programs
transcriptional_programs = miner.read_json(os.path.join(input_path,'GBM\\results\\MINER\\MINER_MicroLowessRNATMM.08.24.2020\\transcriptional_programsmiRNAAndSig.json'))

program_list = [transcriptional_programs[str(key)] for key in range(len(transcriptional_programs.keys()))]

# Create dictionary of program genes
# make dictionary of program keys with gene lists as elements
#pr_genes = {}
#for i in range(len(program_list)):
#    rgns = program_list[i]
#    genes = []
#    for r in rgns:
#        if regulonModules.has_key(r):
#            genes.append(regulonModules[r])
#    genes = list(set(np.hstack(genes)))
#    pr_genes[i] = genes
    

# select reference dictionary for downstream analysis (pr_genes, revisedClusters, coexpressionModules, or regulonModules)
referenceDictionary = pr_genes
# create a background matrix used for statistical hypothesis testing
bkgd = miner.backgroundDf(expressionData)
# for each cluster, give samples that show high coherent cluster activity
overExpressedMembers = miner.biclusterMembershipDictionary(referenceDictionary,bkgd,label=2,p=0.05)
# for each clus|ter, give samples that show low coherent cluster activity
underExpressedMembers = miner.biclusterMembershipDictionary(referenceDictionary,bkgd,label=0,p=0.05)
# convert overExpressedMembers dictionary to binary matrix
overExpressedProgramsMatrix = miner.membershipToIncidence(overExpressedMembers,expressionData)
# convert underExpressedMembers dictionary to binary matrix
underExpressedProgramsMatrix = miner.membershipToIncidence(underExpressedMembers,expressionData)

# Create program matrix with values of {-1,0,1}
dfr_programs = overExpressedProgramsMatrix-underExpressedProgramsMatrix

dfr_programs.to_csv("program_activity_forSigRegulonsMinerGBMRembrandt.csv")

done!
done!


In [12]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 25 10:40:42 2020

@author: mwall
"""
import os
import numpy as np
import pandas as pd
os.chdir('E:\ISB work stuff\Post-Doc_BaligaLab_GBMSYGNALProject')
import miner as miner

# Path to the miner directory
input_path = os.getcwd()
expressionData = pd.read_csv("CCLE_RNAseq_genes_counts_20180929OnlyIDs_TMM.csv", sep=",", index_col=0, header = 0)
# Load resulonModules
regulonModules = miner.read_json(os.path.join(input_path,"GBM\\results\\MINER\\MINER_MicroLowessRNATMM.08.24.2020\\regulons.json"))
regulonDfMicroGbmLatest = pd.read_csv("regulonDfGbmMicroRNASigCoxAndStatSig.csv", header = 0)
regulonDfMicroGbmLatest = list(regulonDfMicroGbmLatest['Regulon_ID'].drop_duplicates())
regulonDfMicroGbmLatest = [str(i) for i in regulonDfMicroGbmLatest]
regulonModules = dict((k, regulonModules[k]) for k in regulonDfMicroGbmLatest if k in regulonModules)
#overExpressedMembersMatrix = pd.read_csv(os.path.join(input_path,"GBM\\results\\MINER\\MINER_MicroLowessRNATMM.08.24.2020\\overExpressedMembersmiRNAAndTFRegulonsSig.csv"),index_col=0,header=0)
#overExpressedMembersMatrix.index = np.array(overExpressedMembersMatrix.index).astype(str)
#underExpressedMembersMatrix = pd.read_csv(os.path.join(input_path,"GBM\\results\\MINER\\MINER_MicroLowessRNATMM.08.24.2020\\underExpressedMembersmiRNAAndTFRegulonsSig.csv"),index_col=0,header=0)
#underExpressedMembersMatrix.index = np.array(underExpressedMembersMatrix.index).astype(str)
#regulonactivity = overExpressedMembersMatrix-underExpressedMembersMatrix
# Load transcriptional programs
transcriptional_programs = miner.read_json(os.path.join(input_path,'GBM\\results\\MINER\\MINER_MicroLowessRNATMM.08.24.2020\\transcriptional_programsmiRNAAndSig.json'))

program_list = [transcriptional_programs[str(key)] for key in range(len(transcriptional_programs.keys()))]

# Create dictionary of program genes
# make dictionary of program keys with gene lists as elements
pr_genes_expanded = {}
for i in range(len(program_list)):
    rgns = program_list[i]
    genes = []
    for r in rgns:
        if r in regulonModules:
            genes.append(r)
    if len(genes) != 0:
        genes = list(set(np.hstack(genes)))
        pr_genes_expanded[i] = genes
# select reference dictionary for downstream analysis (pr_genes, revisedClusters, coexpressionModules, or regulonModules)
referenceDictionary = regulonModules
# create a background matrix used for statistical hypothesis testing
bkgd = miner.backgroundDf(expressionData)
# for each cluster, give samples that show high coherent cluster activity
overExpressedMembers = miner.biclusterMembershipDictionary(referenceDictionary,bkgd,label=2,p=0.05)
# for each clus|ter, give samples that show low coherent cluster activity
underExpressedMembers = miner.biclusterMembershipDictionary(referenceDictionary,bkgd,label=0,p=0.05)
# convert overExpressedMembers dictionary to binary matrix
overExpressedMembersMatrix = miner.membershipToIncidence(overExpressedMembers,expressionData)
overExpressedMembersMatrix.index = np.array(overExpressedMembersMatrix.index).astype(str)
# convert underExpressedMembers dictionary to binary matrix
underExpressedMembersMatrix = miner.membershipToIncidence(underExpressedMembers,expressionData)
underExpressedMembersMatrix.index = np.array(underExpressedMembersMatrix.index).astype(str)

# Create program matrix with values of {-1,0,1}
dfr_regulons = overExpressedMembersMatrix-underExpressedMembersMatrix

overExpressedMembersMatrix.to_csv("CCLE_ActiveRegulons_forAllGbmTCGASigRegulons.csv")

done!
done!


In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 25 10:40:42 2020

@author: mwall
"""
import os
import numpy as np
import pandas as pd
os.chdir('E:\ISB work stuff\Post-Doc_BaligaLab_GBMSYGNALProject')
import miner as miner

# Path to the miner directory
input_path = os.getcwd()
expressionData = pd.read_csv("GbmMicroRNAMergedWithIDsZScored.csv", sep=",", index_col=0, header = 0)
# Load resulonModules
regulonModules = miner.read_json(os.path.join(input_path,"GBM\\results\\MINER\\MINER_MicroLowessRNATMM.08.24.2020\\regulons.json"))
regulonDfMicroGbmLatest = pd.read_csv("regulonDfGbmMicroRNASigCoxAndStatSig.csv", header = 0)
regulonDfMicroGbmLatest = list(regulonDfMicroGbmLatest['Regulon_ID'].drop_duplicates())
regulonDfMicroGbmLatest = [str(i) for i in regulonDfMicroGbmLatest]
regulonModules = dict((k, regulonModules[k]) for k in regulonDfMicroGbmLatest if k in regulonModules)

# Load transcriptional programs
transcriptional_programs = miner.read_json(os.path.join(input_path,'GBM\\results\\MINER\\MINER_MicroLowessRNATMM.08.24.2020\\transcriptional_programsmiRNAAndSig.json'))

program_list = [transcriptional_programs[str(key)] for key in range(len(transcriptional_programs.keys()))]

# Create dictionary of program genes
# make dictionary of program keys with gene lists as elements
pr_genes_expanded = {}
for i in range(len(program_list)):
    rgns = program_list[i]
    genes = []
    for r in rgns:
        if r in regulonModules:
            genes.append(r)
    if len(genes) != 0:
        genes = list(set(np.hstack(genes)))
        pr_genes_expanded[i] = genes

In [4]:
pr_genes_expanded = pd.DataFrame.from_dict(transcriptional_programs, orient='index')

In [8]:
#pr_genes_expanded = pr_genes_expanded.stack()
pr_genes_expanded = pr_genes_expanded.drop_duplicates()

In [10]:
pr_genes_expanded.to_csv("SigRegulonsInPrograms_GBMTCGA.csv")

C:\Users\kkavy\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
